In [1]:
!pip install foolbox[torch] --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.7 MB/s eta 0:00:00


In [5]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/OptiML/repo/OptML-project

/content/drive/MyDrive/OptiML/repo/OptML-project


In [7]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np
import foolbox as fb
from foolbox.attacks import HopSkipJumpAttack, BoundaryAttack
from tqdm import tqdm
import os

In [8]:
#if google drive :
drive_base_path = '/content/drive/MyDrive/OptiML/repo'
cifar10_c_path = os.path.join(drive_base_path, 'CIFAR-10-C')
model_path = os.path.join(drive_base_path, 'OptML-project/Results/SGD/ResNet_Transform_lr_0.05_momentum_0.9.pth')

In [9]:
# STEP 5: Load your model
# Replace with your actual model class
from Functions.implementations import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_resnet18_cifar().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),  # only ToTensor, no normalization!
])

test_dataset = CIFAR10(root="./data", train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [28]:
# Step 4: Wrap the model in Foolbox
mean = np.array([0.4914, 0.4822, 0.4465], dtype=np.float32)
std = np.array([0.2023, 0.1994, 0.2010], dtype=np.float32)

fmodel = fb.PyTorchModel(model, bounds=(0, 1))

# Step 5: Load one image from CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
image, label = dataset[0]
image = image.unsqueeze(0)  # Add batch dimension
label = torch.tensor([label])

print(image.shape)
# Step 6: Run black-box attack
attack = BoundaryAttack()

# Important: HopSkipJumpAttack works in decision-based mode and needs labels
advs, _, success = attack(fmodel, image, label, epsilons = None)

print("Attack success:", success.item())

torch.Size([1, 3, 32, 32])


KeyboardInterrupt: 